In [2]:
from l2r_class import *

/Users/karimasadykova/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Chargement json (Top documents les plus non-pertinents pour chaque requete selon le score BM25)

In [3]:
import json
fichier = 'msmarco-hard-negatives.jsonl'
with open(fichier, 'r') as f:
    first_line = f.readline()
    data = json.loads(first_line)
    print(data['neg']['bm25'])

[{'pid': '6948601', 'bm25-score': 26.068258, 'ce-score': 3.581289291381836}, {'pid': '5129919', 'bm25-score': 26.007563, 'ce-score': 8.257364273071289}, {'pid': '6717931', 'bm25-score': 25.934547, 'ce-score': 8.866464614868164}, {'pid': '1065943', 'bm25-score': 25.258688, 'ce-score': 5.258519172668457}, {'pid': '1626276', 'bm25-score': 25.153275, 'ce-score': 4.19371223449707}, {'pid': '981824', 'bm25-score': 24.716213, 'ce-score': 8.563857078552246}, {'pid': '6449111', 'bm25-score': 24.62115, 'ce-score': 4.907355785369873}, {'pid': '1028927', 'bm25-score': 24.54112, 'ce-score': 7.617893695831299}, {'pid': '2524942', 'bm25-score': 24.355902, 'ce-score': 1.5268436670303345}, {'pid': '5810175', 'bm25-score': 24.315733, 'ce-score': -0.6152520179748535}, {'pid': '6236527', 'bm25-score': 24.3093, 'ce-score': -2.9456772804260254}, {'pid': '7179545', 'bm25-score': 24.130537, 'ce-score': 10.018341064453125}, {'pid': '168979', 'bm25-score': 24.130537, 'ce-score': 10.202350616455078}, {'pid': '15

In [4]:
import json

# Load the JSON data from a file
data_list = []
with open('msmarco-hard-negatives.jsonl', 'r') as file:
    for line in file:
        #print(json.loads(line))
        data_list.append(json.loads(line))
        if len(data_list) > 3:
            break

# Now data_list contains multiple JSON objects, one per line from the file
data_id = []

for data in data_list:
    # Extract positive examples
    pos_examples = [(data['qid'], pos['pid'], 1) for i, pos in enumerate(data['pos']) if i < 1]

    # Extract negative examples with a bm25_score
    neg_examples = [(data['qid'], neg['pid'], 0) for i, neg in enumerate(data['neg']['bm25']) if i < 9]

    # Combine positive and negative examples
    data_id.extend(pos_examples + neg_examples)

# Print the final_data
print(len(data_id))
print(data_id[:20])

40
[('571018', '7349777', 1), ('571018', '6948601', 0), ('571018', '5129919', 0), ('571018', '6717931', 0), ('571018', '1065943', 0), ('571018', '1626276', 0), ('571018', '981824', 0), ('571018', '6449111', 0), ('571018', '1028927', 0), ('571018', '2524942', 0), ('832106', '5090093', 1), ('832106', '5090096', 0), ('832106', '210166', 0), ('832106', '4877671', 0), ('832106', '851808', 0), ('832106', '210158', 0), ('832106', '2492227', 0), ('832106', '3094439', 0), ('832106', '5434531', 0), ('832106', '210165', 0)]


### Chargement MS-Marco

In [5]:
import ir_datasets

# Load the msmarco dataset
dataset = ir_datasets.load('msmarco-passage/train')
store = dataset.docs_store()
print(dataset)

Dataset(id='msmarco-passage/train', provides=['docs', 'queries', 'qrels', 'scoreddocs', 'docpairs'])


In [6]:
dict_query = {}

for query in dataset.queries_iter():
    dict_query[query.query_id] = query.text
    

### Creation Data_NSBM25

In [29]:
import numpy as np

def create_nsbm25():
	# Create an empty list to store the modified data
	data_bm25 = []

	for data in data_id:
		# Extract the values from the original data
		dict_value = dict_query[data[0]]
		store_value = store.get(data[1])[1]
		class_value = data[2]
		
		# Perform one-hot encoding
		class_vector = np.zeros(2)
		class_vector[class_value] = 1
		
		# Create a modified tuple with the encoded class vector
		modified_tuple = (dict_value, store_value, class_vector)
		
		# Append the modified tuple to the list
		data_bm25.append(modified_tuple)
	return data_bm25

data_bm25 = create_nsbm25()
# Print the modified_data
print(len(data_bm25))
print(data_bm25[:5])

40
[('what are the liberal arts?', 'liberal arts. 1. the academic course of instruction at a college intended to provide general knowledge and comprising the arts, humanities, natural sciences, and social sciences, as opposed to professional or technical subjects.', array([0., 1.])), ('what are the liberal arts?', 'The New York State Education Department requires 60 Liberal Arts credits in a Bachelor of Science program and 90 Liberal Arts credits in a Bachelor of Arts program. In the list of course descriptions, courses which are liberal arts for all students are identified by (Liberal Arts) after the course number.', array([1., 0.])), ('what are the liberal arts?', "What Does it Mean to Study Liberal Arts? A liberal arts major offers a broad overview of the arts, sciences, and humanities. Within the context of a liberal arts degree, you can study modern languages, music, English, anthropology, history, women's studies, psychology, math, political science or many other disciplines.", a

### Chargement modele BERT

In [8]:
tokenizer = BertTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")
bert_model = BertModel.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")

Some weights of the model checkpoint at huawei-noah/TinyBERT_General_4L_312D were not used when initializing BertModel: ['fit_denses.3.weight', 'cls.predictions.transform.LayerNorm.weight', 'fit_denses.1.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'fit_denses.2.weight', 'cls.predictions.decoder.weight', 'fit_denses.0.weight', 'fit_denses.3.bias', 'fit_denses.4.weight', 'cls.predictions.transform.dense.bias', 'fit_denses.1.bias', 'cls.seq_relationship.bias', 'fit_denses.4.bias', 'cls.predictions.transform.LayerNorm.bias', 'fit_denses.2.bias', 'cls.predictions.bias', 'fit_denses.0.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing

### LS and T-LS

In [31]:
def learnModeLS(data, num_epochs=1, tls=-1, eps=1e-8):
	dataset = LTRDataset(data, tokenizer, max_len=512)

	# Create data loaders
	loader = DataLoader(dataset, batch_size=16, num_workers=4)

	# Instantiate the model
	model = BertLTRModel(bert_model)

	# Move model to GPU if available
	device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
	model = model.to(device)

	# Define loss function and optimizer
	criterion = nn.CrossEntropyLoss(label_smoothing=eps)
	optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

	list_loss = []
	# Training loop
	for epoch in range(num_epochs):
		model.train()
		total_loss = 0

		for batch in loader:
			# Move batch tensors to the same device as the model 
			input_ids = batch['input_ids'].to(device)
			attention_mask = batch['attention_mask'].to(device)
			relevance = batch['relevance'].to(device)

			# Forward pass
			outputs = model(input_ids=input_ids, attention_mask=attention_mask)

			# Compute loss
			if tls == num_epochs:
				criterion = nn.CrossEntropyLoss()
			loss = criterion(outputs, relevance) 

			# Backward pass and optimize
			optimizer.zero_grad()
			loss.backward()
			optimizer.step()

			total_loss += loss.item()
		loss = total_loss/len(loader)
		list_loss.append(loss)
		print(f'Epoch {epoch}, Loss: {loss}')
	return model, list_loss
	

### Creation Data_NSrandom

In [27]:
import random

def create_nsrandom_samples(data_bm25):
	data_nsrand = []
	queries = [data_bm25[i] for i in range(0, len(data_bm25), 10)]
	docs = {data_bm25[i][1] for i in range(len(data_bm25))}

	for query in queries:
		data_nsrand.append(query)
		chosen_docs = [query[1]]
		for i in range(9):
			doc = random.choice(list(docs))
			while doc in chosen_docs:
				doc = random.choice(list(docs))
			data_nsrand.append((query[0], doc, np.array([1., 0.])))
			chosen_docs.append(doc)
	return data_nsrand

In [28]:
data_nsrand = create_nsrandom_samples(data_bm25)
print(data_nsrand[:10])
print(data_bm25[:10])

[('what are the liberal arts?', 'liberal arts. 1. the academic course of instruction at a college intended to provide general knowledge and comprising the arts, humanities, natural sciences, and social sciences, as opposed to professional or technical subjects.', array([0., 1.])), ('what are the liberal arts?', 'Fibrinolytic Therapy: What You Should Really Know; Fibrinolytic therapy is a crucial medical treatment that makes use of certain special drugs that will work in the body to break down the blood clots blocking the major arteries.ibrinolytic Therapy: What You Should Really Know; Fibrinolytic therapy is a crucial medical treatment that makes use of certain special drugs that will work in the body to break down the blood clots blocking the major arteries.', array([1., 0.])), ('what are the liberal arts?', 'Cardiovascular Disease: Fermented shrimp paste, which is a popular manipulation of the food, contains a fibrinolytic enzyme that can be used for thrombolytic therapy. Thrombolyti

In [32]:
# LS
print('-----LS pour NS_BM25-----')
model_LS_bm25, list_loss_LS_bm25 = learnModelLS(data_bm25, num_epochs=5)
print('-----LS pour NS_rand-----')
model_LS_random, list_loss_LS_random = learnModelLS(data_nsrand, num_epochs=5)
# 10 minutes au total pour 5 queries...

-----LS pour NS_BM25-----
Epoch 0, Loss: 0.680505096912384
Epoch 1, Loss: 0.65514075756073
Epoch 2, Loss: 0.6296883622805277
Epoch 3, Loss: 0.606624960899353
Epoch 4, Loss: 0.5825433929761251
-----LS pour NS_rand-----
Epoch 0, Loss: 0.6774391730626425
Epoch 1, Loss: 0.6488092740376791
Epoch 2, Loss: 0.6229330698649088
Epoch 3, Loss: 0.5986324946085612
Epoch 4, Loss: 0.5777320861816406


In [33]:
# TLS
print('-----TLS pour NS_BM25-----')
model_TLS_bm25, list_loss_TLS_bm25 = learnModelLS(data_bm25, num_epochs=5, tls=2)
print('-----TLS pour NS_rand-----')
model_TLS_random, list_loss_TLS_random = learnModelLS(data_nsrand, num_epochs=5, tls=2)
# 11 minutes au total pour 5 queries...

-----TLS pour NS_BM25-----
Epoch 0, Loss: 0.6661476890246073
Epoch 1, Loss: 0.6397204399108887
Epoch 2, Loss: 0.6150433818499247
Epoch 3, Loss: 0.5928896268208822
Epoch 4, Loss: 0.5708221197128296
-----TLS pour NS_rand-----
Epoch 0, Loss: 0.712973972161611
Epoch 1, Loss: 0.6828835805257162
Epoch 2, Loss: 0.6551411549250284
Epoch 3, Loss: 0.6293860872586569
Epoch 4, Loss: 0.6055442094802856


### WSLS

In [50]:
from wsls_class import *

def learnModelWSLS(data, num_epochs=1, tls=-1, eps=1e-8):
	dataset = LTRDataset(data, tokenizer, max_len=512)

	# Create data loaders
	loader = DataLoader(dataset, batch_size=16, num_workers=4)

	# Instantiate the model
	model = BertLTRModel(bert_model)

	# Move model to GPU if available
	device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
	model = model.to(device)

	# Define loss function and optimizer
	criterion = WeaklySupervisedLabelSmoothingCrossEntropy(label_smoothing=eps)
	optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

	list_loss = []
	# Training loop
	for epoch in range(num_epochs):
		model.train()
		total_loss = 0

		for batch in loader:
			# Move batch tensors to the same device as the model 
			input_ids = batch['input_ids'].to(device)
			attention_mask = batch['attention_mask'].to(device)
			relevance = batch['relevance'].to(device)

			# Forward pass
			outputs = model(input_ids=input_ids, attention_mask=attention_mask)

			# Compute loss
			if tls == num_epochs:
				criterion = nn.CrossEntropyLoss()
			loss = criterion(outputs, relevance) 

			# Backward pass and optimize
			optimizer.zero_grad()
			loss.backward()
			optimizer.step()

			total_loss += loss.item()
		loss = total_loss/len(loader)
		list_loss.append(loss)
		print(f'Epoch {epoch}, Loss: {loss}')
	return model, list_loss

In [51]:
# WSLS
print('-----WSLS pour NS_BM25-----')
model_WSLS_bm25, list_loss_WSLS_bm25 = learnModelWSLS(data_bm25, num_epochs=5)

-----WSLS pour NS_BM25-----


NameError: name 'WeaklySupervisedLabelSmoothingCrossEntropy' is not defined

In [37]:
# WSLS
print('-----WSLS pour NS_rand-----')
model_WSLS_random, list_loss_WSLS_random = learnModelWSLS(data_nsrand, num_epochs=5)

-----WSLS pour NS_rand-----


KeyboardInterrupt: 

### Metrique d'evaluation (Standard Deviation)